In [1]:
import csv
import sys
import math
import os
import librosa
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
sys.setrecursionlimit(10000)


In [2]:
import glob
import os
# reading training data
current_working_directory = os.getcwd()

base_path = current_working_directory + '/CAP6610SP24_training_set/'
prog_path = base_path + 'Progressive_Rock_Songs/'
non_prog_path = base_path + 'Not_Progressive_Rock/'

def fileList(path):
    matches = glob.glob(path + '/**/*', recursive=True)
    files = [file for file in matches if os.path.isfile(file) and file.endswith(('.wav', '.mp3', '.avi', '.flac', '.ogg', '.m4a', '.wma', '.mp4'))]
    return files

prog_files = fileList(prog_path)
non_prog_files = fileList(non_prog_path)

def get_prog_files():
    return prog_files

def get_non_prog_files():
    return non_prog_files

In [3]:
prog_files = get_prog_files()
non_prog_files = get_non_prog_files()

print("Number of Progressive Rock Songs: ", len(prog_files))
print("Number of Non-Progressive Rock Songs: ", len(non_prog_files))

Number of Progressive Rock Songs:  142
Number of Non-Progressive Rock Songs:  359


In [4]:
all_files = prog_files + non_prog_files
fixed_sample_rate = 44100


min_duration = 0
for i in range (len(all_files)):
    y, sr = librosa.load(all_files[i], sr=fixed_sample_rate)
    duration = librosa.get_duration(y=y, sr=sr)
    if i == 0:
        min_duration = duration
    else:
        min_duration = min(min_duration, duration)
min_duration = int(math.floor(min_duration))
print("Minimum Duration: ", min_duration)


Note: Illegal Audio-MPEG-Header 0x4c595249 at offset 20630146.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
Note: Illegal Audio-MPEG-Header 0x4f09842e at offset 3049050.
Note: Trying to resync...
/var/folders/m0/0p062cxn2rvbyqh1m3h8cb4c0000gn/T/ipykernel_75353/4035985224.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(all

Minimum Duration:  37


In [5]:
# feature extraction
min_duration = 30

file = open('training_features_30.csv', 'w', newline='')
header = ['filename','genre', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']

for i in range (1, 21):
    header.append('mfcc'+str(i))

# writing giles to error.txt
error_file = open('error.txt', 'w')
error_file.close()
with file:
    writer = csv.writer(file)
    writer.writerow(header)

genre = "prog"
count = 0
# time_series_length = 30


for i in range (len(prog_files)):
    filename = prog_files[i]
    name = (filename.split('/')[-1]).replace(" ", "_")

    try: 
        y, sr = librosa.load(filename, sr=fixed_sample_rate)
        time = librosa.get_duration(y=y, sr=sr)
        chunks = []
        if time > min_duration:
            org_y = y
            iter = math.floor (time /min_duration)
            print(iter)
            current_size = time*fixed_sample_rate
            chunk_size = min_duration*fixed_sample_rate
            start = 0
            end = chunk_size
            chunk_index = 1

            while iter !=0: 
                count += 1
                chunk = y[start:end]

                chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                rmse = librosa.feature.rms(y=chunk)
                rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y))
                mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                to_append = f'{"prog"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

                # append all the 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'
                
                file = open('training_features_30.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())

                
                # display spectrogram
                
                # spect = librosa.feature.melspectrogram(y=chunk, sr=sr, n_fft=2048, hop_length=512)
                # spect = librosa.power_to_db(spect, ref=np.max)
                # plt.figure(figsize=(12, 8))
                # plit.axis('off')
                # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
                # plt.show()
                
                chunk_index += 1

                start = end
                end = end + chunk_size
                iter -= 1
        else :
            count += 1
            y, sr = librosa.load(filename, sr=fixed_sample_rate, duration=min_duration)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = 10**10*np.mean(librosa.zero_crossings(y)/len(y))
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            to_append = f'{"prog"+name} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

            # append all the 20 rows
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            
            file = open('training_features_30.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

            
            # display spectrogram
            # spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512)
            # spect = librosa.power_to_db(spect, ref=np.max)
            # plt.figure(figsize=(12, 8))
            # plt.axis('off')
            # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
            # plt.show()
    
    except Exception as e:
        print('error handled')
        error_file = open('error.txt', 'a')
        error_file.write(filename + '\n')
        error_file.write(str(e) + '\n')
        error_file.close()
        continue
                    



24
17
17
21
17
14
17
45
15
39
17
33
17
48
24


Note: Illegal Audio-MPEG-Header 0x4c595249 at offset 20630146.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


17
25
13
13
35
41
35
17
15
16
33


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


17
10
45
34
24
17
18
16
51
18
49
28
27
41
20
15
41
7
46
13
87
14
8
8
20
27
22
31


...

12
9
14
25
11
14


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


15
7
31
14
40
10
19
17
9
20
38
20
23
8
15
26
19
43
24
19
21
13
23
41
8
14
22
17
9


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


16
45
11


In [6]:
genre = "non_prog"
# count = 0
# time_series_length = 30


for i in range (len(non_prog_files)):
    filename = non_prog_files[i]
    name = (filename.split('/')[-1]).replace(" ", "_")

    try: 
        y, sr = librosa.load(filename, sr=fixed_sample_rate)
        time = librosa.get_duration(y=y, sr=sr)
        chunks = []
        if time > min_duration:
            org_y = y
            iter = math.floor (time /min_duration)
            print(iter)
            current_size = time*fixed_sample_rate
            chunk_size = min_duration*fixed_sample_rate
            start = 0
            end = chunk_size
            chunk_index = 1

            while iter !=0: 
                count += 1
                chunk = y[start:end]

                chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                rmse = librosa.feature.rms(y=chunk)
                rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y))
                mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                to_append = f'{"nonprog"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

                # append all the 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'
                
                file = open('training_features_30.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())

                
                # display spectrogram
                
                # spect = librosa.feature.melspectrogram(y=chunk, sr=sr, n_fft=2048, hop_length=512)
                # spect = librosa.power_to_db(spect, ref=np.max)
                # plt.figure(figsize=(12, 8))
                # plit.axis('off')
                # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
                # plt.show()
                
                chunk_index += 1

                start = end
                end = end + chunk_size
                iter -= 1
        else :
            count += 1
            y, sr = librosa.load(filename, sr=fixed_sample_rate, duration=min_duration)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = 10**10*np.mean(librosa.zero_crossings(y)/len(y))
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            to_append = f'{"nonprog"+name} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

            # append all the 20 rows
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            
            file = open('training_features_30.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

            
            # display spectrogram
            # spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512)
            # spect = librosa.power_to_db(spect, ref=np.max)
            # plt.figure(figsize=(12, 8))
            # plt.axis('off')
            # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
            # plt.show()
    
    except Exception as e:
        print('error handled')
        error_file = open('error.txt', 'a')
        error_file.write(filename + '\n')
        error_file.write(str(e) + '\n')
        error_file.close()
        continue
                    

6
12
8


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
Note: Illegal Audio-MPEG-Header 0x4f09842e at offset 3049050.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).
/var/folders/m0/0p062cxn2rvbyqh1m3h8cb4c0000gn/T/ipykernel_75353/8423655.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename, sr=fixed_sample_rate)
/Users/harlow/.local/share/virtualenvs/Progect-tV540T77/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


2


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


7
7


...

6
12
8
14
9
10
9
11
9
9
15
8
4
10
7
7
5
5
9
7
10
6
7
8
12
6
7
9


Note: Illegal Audio-MPEG-Header 0x4c595249 at offset 10880470.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


8
10
5
11
16
7
9
6
9
8
6
8
9
6
15
5
10
22
6
8


Note: Illegal Audio-MPEG-Header 0x00000000 at offset 10986313.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


9
9
14
16
10
6
4
4
18
9
6
6
7
14
8
6
15
6
9
7
12
10
8
7
10
